Este notebook teve como objetivo ler uma lista de palavras lexporbr_alfa_excel.xlsx, filtrar de uma forma completamente manual, retornando um json que é utilizado pelo jogo imbroglio.com.br para VALIDAR se uma palavra existe ou não.

Outros notebooks como "conjugando verbos.ipynb" e "Limpando JSON.ipynb" complementam a saida desse notebook 

### Instruções para interpretação do arquivo lexporbr_alfa_excel.xlsx

Segue abaixo o nome, a abreviação e a descrição das 21 colunas de informações apresentadas
nos resultados de uma pesquisa no Léxico do Português Brasileiro, versão Alfa.
Ortografia (orto): forma ortográfica da palavra em letras minúsculas (com exceção dos
nomes próprios), respeitando os acentos específicos de cada palavra37
.
- Categoria gramatical (cat_gram):<
    - categorial gramatical da palavra (adj, adv, gram, nom,num, prop, ver).

- Informação gramatical (inf_gram): 
    - informações gramaticais sobre a palavra (ex. singular/plural, masculino/feminino, passado/presente/futuro, 1/2/3 pessoas, etc.). 
- Frequência ortográfica (freq_orto): 
    - número de vezes que a palavra aparece no NILC (cerca de 32 milhões de palavras).

- Frequência ortográfica por milhão (freq_orto/M): 
     - número de vezes que a palavra aparece entre 1 milhão de palavras. Valor padrão para frequência de palavras.

- Logaritmo natural da frequência ortográfica (log10_freq_orto): 
    - logarítmico natural da frequência ortográfica. Os valores logarítmicos são utilizados para linearizar-se o comportamento das frequências das palavras no corpus.

- Número de letras (nb_letras):
    - número de letras da palavra.

- Número de homógrafas (nb_homogr):
    - número de palavras homógrafas. Palavras que possuem a mesma ortografia ou diferenças de acentos, mas pertencem a categorias gramaticais diferentes.

- Homógrafas (homografas): 
     - categorias gramaticais das palavras homógrafas.

- Ponto de unicidade ortográfico (pu_orto): 
    - letra a partir da qual a palavra se dissocia das outras, ou seja, letra a partir da qual a palavra é única. Sentido da esquerda para direita.

- Vizinhos ortográficos (viz_orto): 
     - número de vizinhos ortográficos a partir do N de Coltheart, ou seja, alterando-se apenas uma letra por vez (Coltheart et al., 1977).

- Distância de Leveinshtein ortográfica (old20):
     - distância ortográfica de Leveinshtein das 20 palavras mais póximas calculadas a partir de regressões lineares (Yarkoni et al., 2008).

- Estrutura CVCV (CVCV_orto):
    - estrutura CVCV da palavra, onde consoantes são C e vogais são V. Ainda, A para acentos, P para pontuação, N para números e S para símbolos.

- Bigramas (bigramas): 
    - bigramas que constituem a palavra separados por “_” e limitados por “#”. O número de bigramas é igual ao número de letras da palavra mais 1.

- Trigramas (trigramas):
     - trigramas que constituem a palavra separados por “_” e limitados por “#”. O número de trigramas é igual ao número de letras da palavra.

- Ortografia invertida (inv_orto):
    - forma invertida da ortografia (orto).

- Estrutura CVCV invertida (inv_CVCV_orto):
    - estrutura CVCV da palavra invertida a partir de (CVCV_orto).

- Bigramas invertidos (inv_bigra): 
     - bigramas que constituem a palavra separados por “_” e limitados por “#” invertidos a partir de (bigramas).

- Trigramas invertidos (inv_trigra):
    - trigramas que constituem a palavra separados por “_” e limitados por “#” invertidos a partir de (trigramas).

- Número aleatório entre 0 e 1 (aleatorio): 
    - número aleatório entre 0 e 1 com oito algarismos de precisão.

- Número de identificação (id): 
    - número de identificação da palavra designado a partir da organização do corpus por frequência decrescente e ordem alfabética a-z. O número de identificação é a posição da palavra no corpus e no léxico


In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re
import json

# Funções

In [ ]:
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                    if unicodedata.category(c) != 'Mn')

In [ ]:
def tratativa(X):  
    '''
    Entrada: series
    X_norm 
        - retira acentos das palavras 
 
    list comprehension
        - remove palavras com hifen
        - remove "."
        - remove palavra com menos de 2 caracter
        - remove palavras com letras maisculas (nomes proprios)
        - remove palavras com caracteres especiais utilizando regex
        - retira palavras escolhidas
        
    return: 2 listas, 1 de palavras sem acentos e outra normal
    '''

    
    #verificacao do regex para caracteres especiais
    string_check = re.compile("[ƒµ‚†„\ˆ¥÷œ‡þ»+'-@_!#$%^&*()<>?/\|}{~:ºª°º¹²³£¢¬]")

    word_list = [x for x in X
                if  '-' not in x 
                and string_check.search(x) == None
                #and len(x) > 2 
                and '.' not in x 
                and x.lower() == x
                ]

    
    word_list_clean = [x for x in word_list if x not in palavroes and x not in norm_palavroes]
        
    
    word_list_clean = [x for x in word_list_clean 
                         if ''.join(
                                 [letter for letter in x 
                                      if not letter.isdigit()]
                                 )
                                ]
    
    word_list_pt = [x for x in word_list_clean if x not in only_en]
    
    # removendo acentos
    y = pd.Series(word_list_pt)
    X_norm = y.apply(lambda row: strip_accents(row))
    
    return X_norm.to_list(), word_list_pt

# IMPORTANDO DICIONARIO PORTUGUES

In [ ]:
PATH = 'Data/dicionario/lexporbr_alfa_excel.xlsx'
lexico = pd.read_excel(PATH)
lexico.head()

In [ ]:
lexico[lexico.cat_gram == 'ver'].sample(10)

# EDA 

In [ ]:
lexico.ortografia.duplicated().sum()

In [ ]:
lexico.ortografia.isna().sum()

# Iniciando Tratativas

### NULOS E DUPLICADAS

In [ ]:
lexico = lexico.dropna(subset=['ortografia'])

In [ ]:
lexico = lexico.drop_duplicates(subset= 'ortografia', keep='first')

-------

## QUANTIDADE DE LETRAS

#### cria dataframe com todas as palavras maiores que 2 caracteres que não sejam numericos

In [ ]:
lexico_filtered = lexico[(lexico['nb_letras'] > 2) & (lexico.cat_gram != 'num')]

##### cria dataframe com todas as palavras de 2 caracteres que não sejam numericos para futuruas tratativas diferenciadas

In [ ]:
lexico_filtered_2_letras = lexico[(lexico['nb_letras'] == 2) & (lexico.cat_gram != 'num')]

In [ ]:
lexico_filtered_2_letras = lexico_filtered_2_letras.sort_values(by='freq_orto/M', ascending=False)

In [ ]:
lexico_filtered_2_letras.shape

In [ ]:
lexico_filtered_2_letras.head()['ortografia']

-------

# LISTA PALAVRAS EM INGLES

#####  lendo dicionario lexico em ingles

In [ ]:
#http://crr.ugent.be/programs-data/subtitle-frequencies
lexico_en = pd.read_excel(r'Data\dicionario\SUBTLEXusfrequencyabove1.xls')
lexico_en.head()

In [ ]:
lexico_en.shape

##### transformando em lista

In [ ]:
english_words = lexico_en.Word.to_list()

##### ordenando pelas mais frequentes (as palavras em pt br ficarão por ultimo, na 6077 colocação) - leitura visual

In [ ]:
br_en = lexico[lexico.ortografia.isin(english_words)].sort_values(by='freq_orto/M').ortografia.to_list()

In [ ]:
br_en[0:20]

##### apos verificação manual, notei que essas palavras são brasileiras (ou frequentemente utilizadas no brasil) 

In [ ]:
list_br ='''exotica, formulas, scan, magazine, erotica, level, karaoke, guacamole, hospital, bug, bugs, cadaver, câncer, videocassete, vina, vodka, trial, loop, loops, protons, pet, fã, delete, kilo, kilos, cameras, camera, supernatural, short, use, megabytes, polyester, tsunami, médico, live, pátio, inglês, vírus, álbum, mimosa, língua, bio, zap, premium, aliens, alien, pet, pets, hétero, áudio, nerds, nerd, panda, pandas, ninja, ninjas, revise, desktop, anime, cloaca, insular, retro, revolver, indie, neural, macular, medicina, multi, multicultural, auras, aura, container, containers, whisky, whiskies, torso, tiara, ultimo, tutu, local, radio, debutante, flamenco, boxer  , box, combine, cacam baste, basta, barmen, ancora, abates, impostor, iguana, zona, senhorita, led, megawatt, familia, arenas, arena, avance, melodrama, network, poodle, script, semi, juris, jambo, ignore, hacker, gurus, cameraman, avatar, motocross, moto, snowboard, placebo, hobbies, hobby, colas, celestial, celeste, cardiovascular, interface, interfaces, cola, elixir, elixires, vulva, tech, taverna, superego, paranormal, tavernas, portifólio, fondue, escudos, clerical, mandala, manda, número, console, jeep, tempera, silos, poster, pixel, pixels, remix, clique, bike, bikes, cliques, samurais, samurai, sonata, banda, bandas, neuroses, peques, rumba, torpedo, tutorial, modular, consular, carte, intima, intimo, intimas, intimos, utublar, tender, sequela, patina, máscara, iguanas, hangar, ferias, edema, descamisados, jumbo, floral, latitude, gourmet, piano, pianos, marrom, marrons, lambada, fax, faxes, surreal, solicitude, critique, date, examine, replay, remove, vendor, mica, pesetas, dolar, tótem, proverbial, fraternal, nasal, intercontinental, paternal, maternal, feedback, crepe, clone, clones, converse, suíte, merengue, ornamental, sapiens, laptop, lustre, ballet, force, cowboy, cetera, bodes, batom, batons, muleta, tango, pulsar, plebe, mambo, pata, garagem, torque, zoom, subverter, facial, larva, margarita, vascular, servo, virtuoso, presto, predecessor, mole, moles, cache, decimal, declare, capô, temperamental, piranha, piranhas, flamingos, abdominal, cabanas, dons, egos, vertebral, posteriori, recuse, diocese, triangular, radial, pólo, mamas, vesceral, eras, manifestos, manifesto, novena, neutrino, colossal, zebras, zebra, doer, ketchup, mini, posses, posse, vagina, menstrual, menstrua, complete, internet, perpendicular, reserve, marque, libido, tacos, taco, magma, umbilical, burros, jingles, provincial, provincia, shop, sensor, poses, fosse, gospel, causa, hematomas, pampas, sbsolve, chef, chefs, exportar, picante, media, aficionados, vans, van, gringo, infame, ocular, salvos, resorte,transistor, tribal, pedestal, anorexia, cone, cones, drinks, drink, pastoral, portal, fórmula, servos, longitudinal, carnal, frame, chassi, visor, limbo, mucosa, aromas, aroma, renal, volts, fetal, homo, concha, genital, spray, hormonal, anal, dorsal, rifle, paparazzi, menu, casco, caviar, manias, salve, fado, lira, haste, logo, logos, expert, loco, loca, compressor, trailer, longitude, sina, imbórglio, grata, altitudes, orbital, rim, skate, champagne, detector, prepare, angular, fractal, executor, modelo, tutor, tapas, video, papas, papa, nuances, manicure, gringos, feudal, você, lava, rea, televisor, mosquitos, mosquito, vocês, feudais, lavas, machismo, trombone, tequila, revive, soda, ardor, enigmas, enigma, sushi, molar, rata, rato, precede, cover, dental, baby, bolos, note, diagonal, salsa, capacitor, odor, secular, pedal, cia, apologia, realize'''
list_br = list_br.split(', ')

In [ ]:
len(list_br)

##### Apos a 6077 palavra, a maioria das palavras estão em portugues, então criei uma lista pra retirar as de ingles entre elas

In [ ]:
list_en = '''hip, flat, club, jet, must, killer, blazer, it, est, kitsch, heavy, new, royalties, remake, you, rockets, slides, he, business, ops, thriller, notes, made, is, hall, es, by, overnight, country, playoffs, franchising, du, th, cr, soft, food, free, drag, out, soccer, at, cap, star, big, flash, rpm, point, notes, seller, best, zapping, ita, ad, off, qua, quas, es, made, is, mote, line, con, grid, besides, look, pool, s, ex, prime, ohms, ram, game, rg, fr, op, cc, ante, on, to, per, et, en, light, y, sex, n, bi, over, of, di, cm, mi, t, in, kg, iii, te, d, 0'''
list_en = list_en.split(', ')
list_en = pd.Series(list_en).apply(lambda row: strip_accents(row))
list_en = list_en.to_list()

In [ ]:
len(list_en)

#### verifica se as primeiras 6077 palavras da lista de palavras "em ingles" não estão na lista de palavras em br e as proxima ESTÃO  lista de palavras ingles

In [ ]:
#retirando as br da lista de ingles
only_en = [x for x in br_en[0:6077] if x not in list_br] + [y for y in br_en[6077:] if y in list_en]

In [ ]:
only_en[0:50]

--------

# PALAVRÕES

#### lendo o ARQUIVO DE PALAVROES 


In [ ]:
#lendo o ARQUIVO DE PALAVROES 
f = open('Data/palavroes.txt') 
palavroes = f.readlines()
f.close()

In [ ]:
palavroes

#### Tratando palavrões

In [ ]:
palavroes = pd.Series(palavroes).apply(lambda x: x.lower().replace('\n ', '').replace(' ', ''))
norm_palavroes = pd.Series(palavroes).apply(lambda row: strip_accents(row))

--------

# CHAMADA DAS FUNÇÕES

In [ ]:
normalized_word_list, word_list= tratativa(lexico_filtered['ortografia'])

In [ ]:
normalized_word_list_2_letras, word_list_2_letras= tratativa(lexico_filtered_2_letras['ortografia'])

In [ ]:
normalized_word_list_2_letras[0:32]

In [ ]:
'jet' in word_list

# Adicionando palavras com 2 letras na lista de palavras totais (normalizadas e nao normalizadas)

In [ ]:
#apos feito uma inspeção visual, foi defido que das 478 palavras de 2 letras, apenas as 32 primeiras sao relevantes. 
#Mesmo com alguns lixos (lista retirar) e de todass as 478 palavras, algumas ainda seriam relevantes (lista inserir e norm_inserir)
retirar = ['ao', 'km', 'ii', 'mg']
inserir = ['lá', 'fé','tú', 'dê', 'se', 'nó', 'dj', 'br', 'pá', 'dó', 'ok','oi', 'nú']
norm_inserir = ['la', 'fe','tu', 'de', 'se', 'no', 'dj', 'br', 'pa', 'do', 'ok','oi', 'nu']


# as 32 primeiras palavras da lista estão ok
#retirnando as palavras normalizadas escolhidas dentre as 32 primeiras da lista de palavras com 2 letras
normalized_word_list_2_letras = [x for x in normalized_word_list_2_letras[0:32] if x not in retirar] 

#inserindo as palavras normalizadas escolhidas
normalized_word_list_2_letras = normalized_word_list_2_letras + norm_inserir

#somando com a lista de palavras normalizadas todais
normalized_word_list = normalized_word_list + normalized_word_list_2_letras


#repetindo o processo para lista de palavras nao normalizadas
#retirnando as palavras escolhidas dentre as 32 primeiras da lista de palavras com 2 letras dentre as 32 primeiras
word_list_2_letras = [x for x in word_list_2_letras[0:32] if x not in retirar] 

#inserindo as palavras escolhidas
word_list_2_letras = word_list_2_letras + norm_inserir

#somando com a lista de palavras
word_list = word_list + word_list_2_letras

In [ ]:
len(word_list)

# CRIANDO O JSON

In [ ]:
import json

i = 0
dic_aux = {}
list_aux = []

for word, norm_word in zip(word_list, normalized_word_list):
    dic_aux = {
        'id': i,
        'word':word,
        'normalized': norm_word
    }    
    list_aux.append(dic_aux)
    i = i+1

## INSERINDO LISTA DE NUMERAIS DIRETO NO JSON

In [ ]:
numerais = 'um, dois, três, quatro, cinco, seis, sete, oito, nove, dez, onze, doze, treze, quatorze, quinze, dezesseis, dezessete, dezoito, dezenove, vinte, trinta, quarenta, cinquenta, sessenta, setenta, oitenta, noventa, cem, duzentos, trezentos, quatrocentos, quinhentos, seiscentos, setecentos, oitocentos, novecentos, mil'
numerais = numerais.split(', ')
norm_numerais = pd.Series(numerais).apply(lambda row: strip_accents(row))


list_aux2 = []
for i in range (len(list_aux), len(list_aux) +  len(numerais)):
    dic_aux = {
        'id': i,
        'word': numerais[i - len(list_aux)],
        'normalized' : norm_numerais.to_list()[i - len(list_aux)]        
    }
    
    list_aux2.append(dic_aux)

In [ ]:
list_aux = list_aux + list_aux2

In [ ]:
len(list_aux)

## SALVANDO O JSON

In [ ]:
#salvando o arquivo json
#with open('Data/words.json', 'w') as f:
#    json.dump(list_aux, f)

----

# RESULTADOS:

In [ ]:
import json
import pandas as pd

#### LENDO O JSON

In [ ]:
#lendo o JSON 
f = open('Data/words.json') 
words_file = json.load(f)
f.close()

In [ ]:
word_list = []
normalized_word_list =[]

for dic in words_file:
    word_list.append(dic['word'])
    normalized_word_list.append(dic['normalized'])

In [ ]:
dic = {'word': word_list,
      'norm_word': normalized_word_list}

In [ ]:
words_df = pd.DataFrame(dic, columns =['word', 'norm_word'])

In [ ]:
words_df

------------